[View in Colaboratory](https://colab.research.google.com/github/fabiansd/Predict-future-sales-Russian-market/blob/master/PricePrediction.ipynb)

In [5]:
!pip install kaggle-cli
!kg download -u fabiansd  -p thisisnotmyrealpassword -c competitive-data-science-predict-future-sales
!gzip -d sales_train.csv.gz
!gzip -d sample_submission.csv.gz
!gzip -d test.csv.gz
!ls

/usr/local/lib/python2.7/dist-packages/mechanicalsoup/browser.py:37: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 37 of the file /usr/local/lib/python2.7/dist-packages/mechanicalsoup/browser.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  response.content, **soup_config)
downloading https://www.kaggle.com/c/competitive-data-science-predict-future-sales/download/item_categories.csv

item_categories.csv 100% |###########################| Time: 0:00:00  10.8 KiB/s

downloading https://www.kaggle.com/c/competitive-data-science-predict-future-sales/download/items.csv

items.csv 100% |#####################################| Time: 0:00:00   2.3 

In [0]:
import pandas as pd
a = pd.read_csv('sales_train.csv')